In [8]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

# === Step 1: 读取并预处理数据 ===
df = pd.read_csv("your_data.csv")  # 替换为你的CSV文件路径
df = df.dropna(subset=["smiles_0", "smiles_1"])

# 分组 - 包含condition_0作为分组依据
unique_combinations = df[['smiles_0', 'smiles_1', 'condition_0']].drop_duplicates().reset_index(drop=True)
unique_combinations['group'] = range(1, len(unique_combinations) + 1)
df = df.merge(unique_combinations, on=['smiles_0', 'smiles_1', 'condition_0'], how='left')

# 将 group 列移到第一列
cols = ['group'] + [col for col in df.columns if col != 'group']
df = df[cols]

df.to_csv('viscosity.csv', index=False)

# 构建 customdata
df['customdata'] = df[['group', 'smiles_0', 'smiles_1', 'concentration_0', 'concentration_1', 'condition_0', 'y_true_0', 'y_pred_0']].to_dict('records')

# === Step 2: 创建图 ===
scatter = go.Scatter(
    x=df['y_true_0'],
    y=df['y_pred_0'],
    mode='markers',
    marker=dict(color='blue', size=8),
    customdata=df['customdata'],
    hoverinfo='none',  # 禁用默认悬停信息
    name="Data"
)

ideal_line = go.Scatter(
    x=[-1, 55],
    y=[-1, 55],
    mode='lines',
    line=dict(dash='dash', color='black'),
    name="y = x"
)

# 设置8:6的宽高比
layout = go.Layout(
    title="Viscosity",
    xaxis=dict(title="y_true_0", range=[-1, 55]),
    yaxis=dict(title="y_pred_0", range=[-1, 55]),
    hovermode='closest',
    width=1200,   # 增加宽度以容纳侧面板
    height=800,  # 设置高度为600像素 (8:6比例)
    margin=dict(l=50, r=50, t=80, b=50)  # 设置边距
)

fig = go.Figure(data=[scatter, ideal_line], layout=layout)

# === Step 3: 生成带自定义 JS 和侧面板的 HTML ===
html_body = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')

# 创建HTML结构和样式
# 创建HTML结构，对CSS中的大括号进行转义
html_structure = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Interactive R² Plot</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        .container {{
            display: flex;
            width: 100%;
        }}
        .plot-container {{
            flex: 3;
        }}
        .info-panel {{
            flex: 1;
            margin-left: 20px;
            padding: 15px;
            border: 1px solid #ccc;
            border-radius: 5px;
            background-color: #f9f9f9;
            height: 700px;
            overflow-y: auto;
        }}
        .info-panel h3 {{
            margin-top: 0;
            border-bottom: 1px solid #ddd;
            padding-bottom: 10px;
        }}
        .info-item {{
            margin-bottom: 5px;
        }}
        .value-display {{
            font-weight: bold;
            color: #2c3e50;
        }}
        .highlight {{
            background-color: #ffeb3b;
            padding: 2px 5px;
            border-radius: 3px;
        }}
        .group-info {{
            background-color: #e3f2fd;
            padding: 10px;
            border-radius: 5px;
            margin-top: 15px;
            border-left: 4px solid #1976d2;
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="plot-container">
            {plot_div}
        </div>
        <div class="info-panel" id="info-panel">
            <h3>数据点信息</h3>
            <p>将鼠标悬停在图表上的点以查看详细信息</p>
            <div class="group-info">
                <div class="info-item">组: <span id="group-value" class="value-display">-</span></div>
                <div class="info-item">条件 (温度): <span id="cond0-value" class="value-display">-</span></div>
            </div>
            <div id="point-info">
                <div class="info-item">真实值: <span id="true-value" class="value-display">-</span></div>
                <div class="info-item">预测值: <span id="pred-value" class="value-display">-</span></div>
                <div class="info-item">第一个组分SMILES: <span id="smiles0-value" class="value-display">-</span></div>
                <div class="info-item">第二个组分SMILES: <span id="smiles1-value" class="value-display">-</span></div>
                <div class="info-item">第一个组分浓度: <span id="conc0-value" class="value-display">-</span></div>
                <div class="info-item">第二个组分浓度: <span id="conc1-value" class="value-display">-</span></div>
            </div>
        </div>
    </div>
    {js_script}
</body>
</html>
"""

# 添加 JS 脚本：hover 时高亮同组点并更新侧面板
js_script = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    // 等待Plotly图表完全加载
    setTimeout(function() {
        const gd = document.querySelector('.js-plotly-plot');
        let hoverTimer = null;
        let currentHoverPoint = null;
        
        // 获取侧面板元素
        const groupValue = document.getElementById('group-value');
        const trueValue = document.getElementById('true-value');
        const predValue = document.getElementById('pred-value');
        const smiles0Value = document.getElementById('smiles0-value');
        const smiles1Value = document.getElementById('smiles1-value');
        const conc0Value = document.getElementById('conc0-value');
        const conc1Value = document.getElementById('conc1-value');
        const cond0Value = document.getElementById('cond0-value');
        
        gd.on('plotly_hover', function(eventData) {
            if (!eventData.points || eventData.points.length === 0) return;
            
            const point = eventData.points[0];
            // 只有第一个trace(散点图)有分组数据
            if (point.curveNumber !== 0) return;
            
            // 清除之前的定时器
            if (hoverTimer) {
                clearTimeout(hoverTimer);
            }
            
            // 保存当前悬停的点
            currentHoverPoint = point;
            
            // 设置0.25秒延迟后高亮
            hoverTimer = setTimeout(function() {
                const pointData = currentHoverPoint.customdata;
                const hoveredGroup = pointData.group;
                const pointCount = gd.data[0].x.length;
                const colors = [];
                
                // 为每个点设置颜色
                for (let i = 0; i < pointCount; i++) {
                    const thisGroup = gd.data[0].customdata[i].group;
                    if (thisGroup === hoveredGroup) {
                        colors.push('red'); // 同一组的点设为红色
                    } else {
                        colors.push('rgba(200, 200, 200, 0.3)'); // 其他组的点设为透明灰色
                    }
                }
                
                // 更新侧面板信息
                groupValue.textContent = pointData.group;
                trueValue.textContent = pointData.y_true_0.toFixed(4);
                predValue.textContent = pointData.y_pred_0.toFixed(4);
                smiles0Value.textContent = pointData.smiles_0;
                smiles1Value.textContent = pointData.smiles_1;
                conc0Value.textContent = pointData.concentration_0;
                conc1Value.textContent = pointData.concentration_1;
                cond0Value.textContent = pointData.condition_0;
                
                // 高亮显示真实值和预测值
                trueValue.classList.add("highlight");
                predValue.classList.add("highlight");
                
                Plotly.restyle(gd, {'marker.color': [colors]}, [0]);
            }, 250); // 设置250毫秒(0.25秒)的延迟
        });
        
        gd.on('plotly_unhover', function() {
            // 清除定时器
            if (hoverTimer) {
                clearTimeout(hoverTimer);
                hoverTimer = null;
            }
            
            // 移除高亮样式
            trueValue.classList.remove("highlight");
            predValue.classList.remove("highlight");
            
            // 恢复所有点的颜色
            const pointCount = gd.data[0].x.length;
            const colors = Array(pointCount).fill('blue');
            Plotly.restyle(gd, {'marker.color': [colors]}, [0]);
        });
    }, 500); // 给予500ms的时间确保Plotly完全加载
});
</script>
"""

# 合并 HTML
full_html = html_structure.format(plot_div=html_body, js_script=js_script)

# === Step 4: 保存为 HTML 文件 ===
with open("viscosity.html", "w", encoding="utf-8") as f:
    f.write(full_html)



In [ ]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

# === Step 1: 读取并预处理数据 ===
df = pd.read_csv("results_epsilon.csv")  # 替换为你的CSV文件路径
df = df.dropna(subset=["smiles_0", "smiles_1"])

# 分组 - 包含condition_0作为分组依据
unique_combinations = df[['smiles_0', 'smiles_1', 'condition_0']].drop_duplicates().reset_index(drop=True)
unique_combinations['group'] = range(1, len(unique_combinations) + 1)
df = df.merge(unique_combinations, on=['smiles_0', 'smiles_1', 'condition_0'], how='left')

# 将 group 列移到第一列
cols = ['group'] + [col for col in df.columns if col != 'group']
df = df[cols]

# 构建 customdata
df['customdata'] = df[['group', 'smiles_0', 'smiles_1', 'concentration_0', 'concentration_1', 'condition_0', 'y_true_0', 'y_pred_0']].to_dict('records')

# === Step 2: 创建图 ===
scatter = go.Scatter(
    x=df['y_true_0'],
    y=df['y_pred_0'],
    mode='markers',
    marker=dict(color='blue', size=8),
    customdata=df['customdata'],
    hoverinfo='none',  # 禁用默认悬停信息
    name="Data"
)

ideal_line = go.Scatter(
    x=[-5, 200],
    y=[-5, 200],
    mode='lines',
    line=dict(dash='dash', color='black'),
    name="y = x"
)

# 设置8:6的宽高比
layout = go.Layout(
    title="Epsilon",
    xaxis=dict(title="y_true_0", range=[-5, 200]),
    yaxis=dict(title="y_pred_0", range=[-5, 200]),
    hovermode='closest',
    width=1200,   # 增加宽度以容纳侧面板
    height=800,  # 设置高度为600像素 (8:6比例)
    margin=dict(l=50, r=50, t=80, b=50)  # 设置边距
)

fig = go.Figure(data=[scatter, ideal_line], layout=layout)

# === Step 3: 生成带自定义 JS 和侧面板的 HTML ===
html_body = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')

# 创建HTML结构和样式
# 创建HTML结构，对CSS中的大括号进行转义
html_structure = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Interactive R² Plot</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        .container {{
            display: flex;
            width: 100%;
        }}
        .plot-container {{
            flex: 3;
        }}
        .info-panel {{
            flex: 1;
            margin-left: 20px;
            padding: 15px;
            border: 1px solid #ccc;
            border-radius: 5px;
            background-color: #f9f9f9;
            height: 700px;
            overflow-y: auto;
        }}
        .info-panel h3 {{
            margin-top: 0;
            border-bottom: 1px solid #ddd;
            padding-bottom: 10px;
        }}
        .info-item {{
            margin-bottom: 5px;
        }}
        .value-display {{
            font-weight: bold;
            color: #2c3e50;
        }}
        .highlight {{
            background-color: #ffeb3b;
            padding: 2px 5px;
            border-radius: 3px;
        }}
        .group-info {{
            background-color: #e3f2fd;
            padding: 10px;
            border-radius: 5px;
            margin-top: 15px;
            border-left: 4px solid #1976d2;
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="plot-container">
            {plot_div}
        </div>
        <div class="info-panel" id="info-panel">
            <h3>数据点信息</h3>
            <p>将鼠标悬停在图表上的点以查看详细信息</p>
            <div class="group-info">
                <div class="info-item">组: <span id="group-value" class="value-display">-</span></div>
                <div class="info-item">条件 (温度): <span id="cond0-value" class="value-display">-</span></div>
            </div>
            <div id="point-info">
                <div class="info-item">真实值: <span id="true-value" class="value-display">-</span></div>
                <div class="info-item">预测值: <span id="pred-value" class="value-display">-</span></div>
                <div class="info-item">第一个组分SMILES: <span id="smiles0-value" class="value-display">-</span></div>
                <div class="info-item">第二个组分SMILES: <span id="smiles1-value" class="value-display">-</span></div>
                <div class="info-item">第一个组分浓度: <span id="conc0-value" class="value-display">-</span></div>
                <div class="info-item">第二个组分浓度: <span id="conc1-value" class="value-display">-</span></div>
            </div>
        </div>
    </div>
    {js_script}
</body>
</html>
"""

# 添加 JS 脚本：hover 时高亮同组点并更新侧面板
js_script = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    // 等待Plotly图表完全加载
    setTimeout(function() {
        const gd = document.querySelector('.js-plotly-plot');
        let hoverTimer = null;
        let currentHoverPoint = null;
        
        // 获取侧面板元素
        const groupValue = document.getElementById('group-value');
        const trueValue = document.getElementById('true-value');
        const predValue = document.getElementById('pred-value');
        const smiles0Value = document.getElementById('smiles0-value');
        const smiles1Value = document.getElementById('smiles1-value');
        const conc0Value = document.getElementById('conc0-value');
        const conc1Value = document.getElementById('conc1-value');
        const cond0Value = document.getElementById('cond0-value');
        
        gd.on('plotly_hover', function(eventData) {
            if (!eventData.points || eventData.points.length === 0) return;
            
            const point = eventData.points[0];
            // 只有第一个trace(散点图)有分组数据
            if (point.curveNumber !== 0) return;
            
            // 清除之前的定时器
            if (hoverTimer) {
                clearTimeout(hoverTimer);
            }
            
            // 保存当前悬停的点
            currentHoverPoint = point;
            
            // 设置0.25秒延迟后高亮
            hoverTimer = setTimeout(function() {
                const pointData = currentHoverPoint.customdata;
                const hoveredGroup = pointData.group;
                const pointCount = gd.data[0].x.length;
                const colors = [];
                
                // 为每个点设置颜色
                for (let i = 0; i < pointCount; i++) {
                    const thisGroup = gd.data[0].customdata[i].group;
                    if (thisGroup === hoveredGroup) {
                        colors.push('red'); // 同一组的点设为红色
                    } else {
                        colors.push('rgba(200, 200, 200, 0.3)'); // 其他组的点设为透明灰色
                    }
                }
                
                // 更新侧面板信息
                groupValue.textContent = pointData.group;
                trueValue.textContent = pointData.y_true_0.toFixed(4);
                predValue.textContent = pointData.y_pred_0.toFixed(4);
                smiles0Value.textContent = pointData.smiles_0;
                smiles1Value.textContent = pointData.smiles_1;
                conc0Value.textContent = pointData.concentration_0;
                conc1Value.textContent = pointData.concentration_1;
                cond0Value.textContent = pointData.condition_0;
                
                // 高亮显示真实值和预测值
                trueValue.classList.add("highlight");
                predValue.classList.add("highlight");
                
                Plotly.restyle(gd, {'marker.color': [colors]}, [0]);
            }, 250); // 设置250毫秒(0.25秒)的延迟
        });
        
        gd.on('plotly_unhover', function() {
            // 清除定时器
            if (hoverTimer) {
                clearTimeout(hoverTimer);
                hoverTimer = null;
            }
            
            // 移除高亮样式
            trueValue.classList.remove("highlight");
            predValue.classList.remove("highlight");
            
            // 恢复所有点的颜色
            const pointCount = gd.data[0].x.length;
            const colors = Array(pointCount).fill('blue');
            Plotly.restyle(gd, {'marker.color': [colors]}, [0]);
        });
    }, 500); // 给予500ms的时间确保Plotly完全加载
});
</script>
"""

# 合并 HTML
full_html = html_structure.format(plot_div=html_body, js_script=js_script)

# === Step 4: 保存为 HTML 文件 ===
with open("epsilon.html", "w", encoding="utf-8") as f:
    f.write(full_html)



In [9]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
import json # 导入json库

# --- Step 1: 读取并预处理数据 ---
# 确保替换为你的CSV文件路径
try:
    df = pd.read_csv("your_data.csv")
except FileNotFoundError:
    print("错误：找不到 your_data.csv 文件。请确保文件存在并路径正确。")
    exit()

df = df.dropna(subset=["smiles_0", "smiles_1", "y_true_0", "y_pred_0", "condition_0", "concentration_0", "concentration_1"]) # 添加更多必须存在的列检查

# 分组 - 包含condition_0作为分组依据
# 使用 factorize 创建更紧凑的 group ID，并保留原始顺序
df['group'], unique_combinations_index = pd.factorize(df[['smiles_0', 'smiles_1', 'condition_0']].apply(tuple, axis=1))
df['group'] = df['group'] + 1 # 从1开始编号

# 将 group 列移到第一列
cols = ['group'] + [col for col in df.columns if col != 'group']
df = df[cols]

# 保存处理后的数据到 viscosity.csv
# 确保包含所有需要的列，包括 group
df.to_csv('viscosity.csv', index=False)

# 将数据转换为适合JS DataTables使用的JSON格式 (records orientation)
# 并包含原始DataFrame的索引，以便于 Plotly 点和表格行之间的映射
df['original_index'] = df.index # 添加原始索引列
data_for_datatables = df.to_dict('records') # list of dictionaries
json_data_string = json.dumps(data_for_datatables, indent=None) # Convert to JSON string

# 构建 customdata (Plotly hover info)
# 确保 customdata 的顺序与 data_for_datatables 中的列一致，
# 并包含 original_index 用于 Plotly 点到 DataTables 行的映射
df['customdata'] = df[['original_index', 'group', 'smiles_0', 'smiles_1', 'concentration_0', 'concentration_1', 'condition_0', 'y_true_0', 'y_pred_0']].values.tolist()


# === Step 2: 创建图 ===
scatter = go.Scattergl( # 使用 Scattergl 提高性能
    x=df['y_true_0'],
    y=df['y_pred_0'],
    mode='markers',
    marker=dict(color='blue', size=8, opacity=0.6), # 添加透明度
    customdata=df['customdata'],
    hoverinfo='none',  # 禁用默认悬停信息
    name="数据点"
)

ideal_line = go.Scattergl( # 使用 Scattergl
    x=[-1, 55],
    y=[-1, 55],
    mode='lines',
    line=dict(dash='dash', color='black'),
    name="y = x"
)

# 设置布局
layout = go.Layout(
    title="粘度预测结果",
    xaxis=dict(title="真实值 (y_true_0)", range=[-1, 55]),
    yaxis=dict(title="预测值 (y_pred_0)", range=[-1, 55]),
    hovermode='closest',
    # 调整宽度以容纳侧面板，高度为图表高度，表格将独立滚动
    width=1200,
    height=600, # Plotly图表的高度
    margin=dict(l=50, r=50, t=80, b=50),
    showlegend=True,
    legend=dict(x=1, y=1)
)

fig = go.Figure(data=[scatter, ideal_line], layout=layout)

# === Step 3: 生成带自定义 JS 和侧面板、表格的 HTML ===
# 获取 Plotly 图表的 div HTML
plot_div_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')

# 创建HTML结构和样式
# 注意：CSS中的大括号 {{ 和 }} 需要转义
html_structure = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>交互式预测结果分析</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.11.5/css/jquery.dataTables.css">
    <script type="text/javascript" charset="utf8" src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script type="text/javascript" charset="utf8" src="https://cdn.datatables.net/1.11.5/js/jquery.dataTables.js"></script>
    <style>
        body {{
            font-family: sans-serif;
            line-height: 1.6;
            margin: 20px;
        }}
        .container {{
            display: flex;
            width: 100%;
            margin-bottom: 20px; /* Add space between plot section and table section */
        }}
        .plot-container {{
            flex: 3;
            min-width: 0; /* Allow flex item to shrink */
            box-sizing: border-box;
            padding-right: 20px; /* Space between plot and panel */
        }}
        .info-panel {{
            flex: 1;
            min-width: 250px; /* Minimum width for panel */
            max-width: 350px; /* Maximum width for panel */
            padding: 15px;
            border: 1px solid #ccc;
            border-radius: 5px;
            background-color: #f9f9f9;
            height: 600px; /* Match plot height */
            overflow-y: auto; /* Make info panel scrollable if content exceeds height */
            box-sizing: border-box;
        }}
        .info-panel h3 {{
            margin-top: 0;
            border-bottom: 1px solid #ddd;
            padding-bottom: 10px;
            margin-bottom: 15px;
            color: #333;
        }}
        .info-item {{
            margin-bottom: 8px;
            font-size: 0.95em;
        }}
        .value-display {{
            font-weight: bold;
            color: #0056b3; /* A shade of blue */
            word-break: break-word; /* Break long SMILES strings */
        }}
        .highlight {{
            background-color: #ffeb3b; /* Yellow highlight */
            padding: 2px 5px;
            border-radius: 3px;
        }}
        .group-info {{
            background-color: #e3f2fd; /* Light blue background */
            padding: 10px;
            border-radius: 5px;
            margin-top: 15px;
            margin-bottom: 15px;
            border-left: 4px solid #2196f3; /* Blue border */
        }}
        .table-section {{
            margin-top: 20px;
        }}
        .table-header {{
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 10px;
        }}
        .table-container {{
             /* DataTables handles internal scrolling via scrollY option */
        }}
        #dataTable {{
            width: 100% !important; /* Ensure DataTables uses full width */
        }}
        #dataTable tbody tr.highlighted-row {{
            background-color: #fff9c4 !important; /* Light yellow for table row highlight */
            font-weight: bold;
        }}
        #download-csv {{
            padding: 8px 15px;
            font-size: 1em;
            color: #fff;
            background-color: #28a745; /* Green */
            border: none;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.3s ease;
        }}
        #download-csv:hover {{
            background-color: #218838;
        }}
    </style>
</head>
<body>
    <div class="main-container">
        <div class="container"> <div class="plot-container">
                {plot_div}
            </div>
            <div class="info-panel" id="info-panel">
                <h3>数据点详细信息</h3>
                <p>将鼠标悬停在图表上的点或点击表格行以查看详细信息。</p>
                <div class="group-info">
                    <div class="info-item">组编号: <span id="group-value" class="value-display">-</span></div>
                    <div class="info-item">条件 (condition_0): <span id="cond0-value" class="value-display">-</span></div>
                </div>
                <div id="point-info">
                    <div class="info-item">真实值 (y_true_0): <span id="true-value" class="value-display">-</span></div>
                    <div class="info-item">预测值 (y_pred_0): <span id="pred-value" class="value-display">-</span></div>
                    <div class="info-item">组分 0 SMILES: <span id="smiles0-value" class="value-display">-</span></div>
                    <div class="info-item">组分 1 SMILES: <span id="smiles1-value" class="value-display">-</span></div>
                    <div class="info-item">组分 0 浓度: <span id="conc0-value" class="value-display">-</span></div>
                    <div class="info-item">组分 1 浓度: <span id="conc1-value" class="value-display">-</span></div>
                </div>
            </div>
        </div>

        <div class="table-section"> <div class="table-header">
                <h3>数据表格</h3>
                <a href="viscosity.csv" download="viscosity.csv">
                    <button id="download-csv">下载 CSV</button>
                </a>
            </div>
            <div class="table-container">
                <table id="dataTable" class="display compact">
                    <thead>
                        <tr>
                            <th>原始索引</th>
                            <th>组编号</th>
                            <th>条件 (condition_0)</th>
                            <th>组分 0 SMILES</th>
                            <th>组分 1 SMILES</th>
                            <th>组分 0 浓度</th>
                            <th>组分 1 浓度</th>
                            <th>真实值 (y_true_0)</th>
                            <th>预测值 (y_pred_0)</th>
                            </tr>
                    </thead>
                    <tbody>
                        </tbody>
                </table>
            </div>
        </div>
    </div>

    {js_script}

</body>
</html>
"""

# 添加 JS 脚本：Plotly hover/unhover, Table click, DataTables 初始化, 联动逻辑
js_script = """
<script>
$(document).ready(function() {{ // Use jQuery ready for DataTables initialization
    // Embed JSON data safely within the script
    const rawJsonDataString = "{json_data_string}";
    let data = [];
    try {{
        data = JSON.parse(rawJsonDataString);
    }} catch (e) {{
        console.error("Failed to parse JSON data:", e);
        // Optionally display an error message to the user
        return; // Stop script execution if data loading fails
    }}

    const gd = document.querySelector('.js-plotly-plot');
    if (!gd) {{
        console.error("Plotly plot element not found.");
        // Optionally display an error message
        return;
    }}

    let hoverTimer = null;
    let currentHoverPointIndex = null; // Store the index of the currently hovered point

    // Get info panel elements
    const groupValue = $('#group-value');
    const trueValue = $('#true-value');
    const predValue = $('#pred-value');
    const smiles0Value = $('#smiles0-value');
    const smiles1Value = $('#smiles1-value');
    const conc0Value = $('#conc0-value');
    const conc1Value = $('#conc1-value');
    const cond0Value = $('#cond0-value');
    const pointInfoDiv = $('#point-info');

    // Initialize DataTable
    const dataTable = $('#dataTable').DataTable({{
        data: data,
        columns: [
            {{ data: 'original_index', title: '原始索引' }},
            {{ data: 'group', title: '组编号' }},
            {{ data: 'condition_0', title: '条件 (condition_0)' }},
            {{ data: 'smiles_0', title: '组分 0 SMILES' }},
            {{ data: 'smiles_1', title: '组分 1 SMILES' }},
            {{ data: 'concentration_0', title: '组分 0 浓度' }},
            {{ data: 'concentration_1', title: '组分 1 浓度' }},
            {{ data: 'y_true_0', title: '真实值 (y_true_0)', render: $.fn.dataTable.render.number(',', '.', 4) }}, // Format number
            {{ data: 'y_pred_0', title: '预测值 (y_pred_0)', render: $.fn.dataTable.render.number(',', '.', 4) }}  // Format number
        ],
        scrollY: '400px', // Make table body scrollable with fixed height
        scrollCollapse: true, // Allow table to shrink below scrollY if data fits
        paging: false, // Disable pagination
        info: false, // Disable "Showing X of Y entries" info
        filter: true, // Enable search filter
        ordering: true, // Enable sorting
        order: [[ 1, 'asc' ]], // Default sort by group column
        columnDefs: [
            {{
                targets: [0], // Hide the original index column by default
                visible: false,
                searchable: false
            }}
        ]
    }});

    // Function to update the info panel
    function updateInfoPanel(pointData) {{
        groupValue.text(pointData.group);
        trueValue.text(pointData.y_true_0.toFixed(4));
        predValue.text(pointData.y_pred_0.toFixed(4));
        smiles0Value.text(pointData.smiles_0);
        smiles1Value.text(pointData.smiles_1);
        conc0Value.text(pointData.concentration_0);
        conc1Value.text(pointData.concentration_1);
        cond0Value.text(pointData.condition_0);

        // Add highlight class
        trueValue.addClass("highlight");
        predValue.addClass("highlight");
    }}

    // Function to clear info panel
    function clearInfoPanel() {{
        groupValue.text('-');
        trueValue.text('-');
        predValue.text('-');
        smiles0Value.text('-');
        smiles1Value.text('-');
        conc0Value.text('-');
        conc1Value.text('-');
        cond0Value.text('-');

        // Remove highlight class
        trueValue.removeClass("highlight");
        predValue.removeClass("highlight");
    }}

    // Function to highlight points on the plot by group
    function highlightPlotPointsByGroup(groupId) {{
        const pointCount = gd.data[0].x.length;
        const colors = [];
        const originalColors = Array(pointCount).fill('blue'); // Default color
        const dimmedColor = 'rgba(200, 200, 200, 0.3)'; // Dimmed color
        const highlightColor = 'red'; // Highlight color

        if (groupId === null) {{ // If no group is selected/hovered
            Plotly.restyle(gd, {{'marker.color': [originalColors]}}, [0]);
        }} else {{
            // Determine colors based on group
            for (let i = 0; i < pointCount; i++) {{
                 // customdata structure: [original_index, group, smiles0, smiles1, conc0, conc1, cond0, ytrue0, ypred0]
                const pointGroup = gd.data[0].customdata[i][1];
                colors.push(pointGroup === groupId ? highlightColor : dimmedColor);
            }}
            Plotly.restyle(gd, {{'marker.color': [colors]}}, [0]);
        }}
    }}

     // Function to highlight table rows by group
    function highlightTableRowsByGroup(groupId) {{
        // Clear previous highlights
        dataTable.$('tr.highlighted-row').removeClass('highlighted-row');

        if (groupId !== null) {{
             // Find rows with matching group ID
            dataTable.rows(function(idx, rowData, node) {{
                return rowData.group === groupId;
            }}).nodes().to$().addClass('highlighted-row');
        }}
    }}

    // Function to scroll table to a specific row (by original index)
    function scrollToRow(originalIndex) {{
        const row = dataTable.rows(function(idx, rowData, node) {{
            return rowData.original_index === originalIndex;
        }});
        if (row.any()) {{
            const node = row.node();
            if (node) {{
                node.scrollIntoView({{ behavior: 'smooth', block: 'nearest' }});
            }}
        }}
    }}


    // --- Plotly Hover/Unhover Events ---
    gd.on('plotly_hover', function(eventData) {{
        if (!eventData.points || eventData.points.length === 0) return;

        // Only process points from the first trace (scatter plot)
        const point = eventData.points[0];
        if (point.curveNumber !== 0) return;

        // customdata structure: [original_index, group, smiles0, smiles1, conc0, conc1, cond0, ytrue0, ypred0]
        const pointData = point.customdata;
        const originalIndex = pointData[0];
        const hoveredGroup = pointData[1];

        // Clear previous timer
        if (hoverTimer) {{
            clearTimeout(hoverTimer);
        }}

        // Save current point index
        currentHoverPointIndex = originalIndex;

        // Set a delay before highlighting and updating to prevent flickering on quick moves
        hoverTimer = setTimeout(function() {{
            if (currentHoverPointIndex === originalIndex) {{ // Ensure we are still hovering the same point
                // Update info panel
                updateInfoPanel({{
                     group: pointData[1],
                     smiles_0: pointData[2],
                     smiles_1: pointData[3],
                     concentration_0: pointData[4],
                     concentration_1: pointData[5],
                     condition_0: pointData[6],
                     y_true_0: pointData[7],
                     y_pred_0: pointData[8]
                }});

                // Highlight plot points of the same group
                highlightPlotPointsByGroup(hoveredGroup);

                // Highlight table rows of the same group and scroll to the hovered point's row
                highlightTableRowsByGroup(hoveredGroup);
                scrollToRow(originalIndex);
            }}
        }}, 100); // 100ms delay
    }});

    gd.on('plotly_unhover', function() {{
        // Clear the timer and the stored point index
        if (hoverTimer) {{
            clearTimeout(hoverTimer);
        }}
        currentHoverPointIndex = null;

        // Reset plot point colors and table highlights after a short delay,
        // but only if no other point is being hovered (handled by timer logic)
        // Or, simply clear on unhover regardless, the next hover will re-highlight.
        // Let's clear immediately for simplicity.
        highlightPlotPointsByGroup(null); // Reset plot colors
        highlightTableRowsByGroup(null); // Clear table highlights
        clearInfoPanel(); // Clear info panel
    }});


    // --- DataTables Row Click Event ---
    $('#dataTable tbody').on('click', 'tr', function() {{
        // Clear any active hover timer from Plotly
        if (hoverTimer) {{
            clearTimeout(hoverTimer);
            currentHoverPointIndex = null;
        }}

        // Remove highlight from previously clicked row(s)
        // dataTable.$('tr.highlighted-row').removeClass('highlighted-row');
        // The highlightTableRowsByGroup function already clears previous highlights

        const rowData = dataTable.row(this).data();

        if (rowData) {{
             // Highlight the clicked row(s) in the table and others in the same group
             highlightTableRowsByGroup(rowData.group);

             // Update info panel with clicked row data
             updateInfoPanel(rowData);

             // Highlight corresponding points on the plot by group
             highlightPlotPointsByGroup(rowData.group);

             // Scroll plot into view if needed? Maybe not necessary, table is below.
        }}
    }});

    // Optional: Add a way to unselect/clear highlights by clicking outside points/table
    // For example, clicking on the plot background or table background could clear highlights
    gd.addEventListener('click', function(event) {{
        // Check if the click was NOT on a data point (e.g., on the plot background)
        // This is a bit tricky with Plotly, plotly_click event might be better,
        // but it only fires on points. A simpler approach is to rely on unhover
        // or explicitly add a "clear" button.
        // Let's use a timeout after click processing to potentially clear if no point was clicked.
        // This is complex, sticking to hover-out and clicking a different row is simpler for now.
    }});

    // Initial state: clear info panel and set default plot colors
    clearInfoPanel();
    highlightPlotPointsByGroup(null);

    // Ensure Plotly is responsive and redraws if parent container resizes
    // (Might need a window resize listener if the layout is highly dynamic,
    // but flexbox handles basic resizing well)
}});
</script>
"""

# Escape the JSON string for embedding within the JavaScript string literal
# Replace backslashes first, then double quotes. This order is important.
# Or, use json.dumps directly which escapes internal quotes correctly,
# and then wrap the resulting string in a JS string literal.
# Let's use json.dumps output directly wrapped in single quotes for robustness,
# as JSON uses double quotes internally.
# However, if a data value contains a single quote, this will break.
# Using double quotes for the JS literal and escaping internal double quotes from json.dumps is safer.
# json.dumps already escapes internal double quotes as \". So we just need to ensure the outer string literal is correct.
# Let's wrap the json_data_string in double quotes in JS, and json.dumps output is already correct.

json_data_string_for_js = json.dumps(data_for_datatables, indent=None) # Get the JSON string

# Inject the plot HTML and the JS script (with embedded JSON data) into the main structure
# Need to be careful with single/double quotes if json_data_string_for_js contains them.
# Using triple quotes for the python string might help readability, but escaping is key for the JS literal.
# Let's use f-string like formatting after defining the template, which is safer for embedding strings.

# Reformat the html_structure and js_script to use f-string placeholders if needed,
# or stick to .format and ensure the JSON string is correctly escaped for the JS literal.
# Let's stick to .format for consistency with the original code structure.
# The safest way is to ensure the string inserted into `"{...}"` in JS is properly escaped for JS.
# json.dumps output is safe for JSON, but needs escaping for a *JS string literal*.
# Example: JSON string `{"name": "O'Reilly"}` becomes JS literal `'{"name": "O\'Reilly"}'`.
# json.dumps gives `{"name": "O'Reilly"}`. If using `"{}"`.format(json_string), it works if json_string has no "
# If using `'{}'`.format(json_string), it works if json_string has no '.
# Since JSON uses " for keys/strings, embedding in " is risky. Embedding in ' is better,
# but need to escape ' within the JSON.

# Let's assume the data doesn't contain characters that will break embedding this way,
# as json.dumps escapes internal double quotes as \". We will wrap the string in `JSON.parse('...')`
# in JS, which means we need to escape any single quotes (') and backslashes (\) in the JSON string.

# The most robust way without complex templating is:
# 1. Get the JSON string: `json_data_string = json.dumps(data_for_datatables)`
# 2. Escape it for embedding in a JavaScript string literal enclosed in single quotes:
#    `escaped_json_string = json_data_string.replace('\\', '\\\\').replace("'", "\\'")`
# 3. In the JS script, use `const rawJsonDataString = '{}'.replace(/\\\\/g, '\\').replace(/\\'/g, "'");`. This is reversed escaping logic...
#    A simpler way: `const rawJsonDataString = `{escaped_json_string}`;` (using template literals in JS)
#    Let's use template literals in JS and ensure Python formatting correctly inserts the string.

# Python side:
json_data_string = json.dumps(data_for_datatables, indent=None)

# JS side template:
# const rawJsonDataString = `{json_data_string_placeholder}`; // JS template literal needs backticks
# const data = JSON.parse(rawJsonDataString);

# Let's refine the Python formatting.
# Use a placeholder directly in the JS script that Python will replace.
js_script_template = """
<script>
$(document).ready(function() {{
    // Embed JSON data
    const data = {json_data_placeholder}; // Python will inject the JSON string here directly

    const gd = document.querySelector('.js-plotly-plot');
    if (!gd) {{
        console.error("Plotly plot element not found.");
        return;
    }}

    let hoverTimer = null;
    let currentHoverPointIndex = null;

    // Get info panel elements
    const groupValue = $('#group-value');
    const trueValue = $('#true-value');
    const predValue = $('#pred-value');
    const smiles0Value = $('#smiles0-value');
    const smiles1Value = $('#smiles1-value');
    const conc0Value = $('#conc0-value');
    const conc1Value = $('#conc1-value');
    const cond0Value = $('#cond0-value');

    // Initialize DataTable
    const dataTable = $('#dataTable').DataTable({{
        data: data,
        columns: [
            {{ data: 'original_index', title: '原始索引' }},
            {{ data: 'group', title: '组编号' }},
            {{ data: 'condition_0', title: '条件 (condition_0)' }},
            {{ data: 'smiles_0', title: '组分 0 SMILES' }},
            {{ data: 'smiles_1', title: '组分 1 SMILES' }},
            {{ data: 'concentration_0', title: '组分 0 浓度' }},
            {{ data: 'concentration_1', title: '组分 1 浓度' }},
            {{ data: 'y_true_0', title: '真实值 (y_true_0)', render: $.fn.dataTable.render.number(',', '.', 4) }},
            {{ data: 'y_pred_0', title: '预测值 (y_pred_0)', render: $.fn.dataTable.render.number(',', '.', 4) }}
        ],
        scrollY: '400px',
        scrollCollapse: true,
        paging: false,
        info: false,
        filter: true,
        ordering: true,
        order: [[ 1, 'asc' ]],
        columnDefs: [
            {{ targets: [0], visible: false, searchable: false }}
        ]
    }});

    // Function to update the info panel
    function updateInfoPanel(pointData) {{
        groupValue.text(pointData.group);
        trueValue.text(pointData.y_true_0.toFixed(4));
        predValue.text(pointData.y_pred_0.toFixed(4));
        smiles0Value.text(pointData.smiles_0);
        smiles1Value.text(pointData.smiles_1);
        conc0Value.text(pointData.concentration_0);
        conc1Value.text(pointData.concentration_1);
        cond0Value.text(pointData.condition_0);

        trueValue.addClass("highlight");
        predValue.addClass("highlight");
    }}

    // Function to clear info panel
    function clearInfoPanel() {{
        groupValue.text('-');
        trueValue.text('-');
        predValue.text('-');
        smiles0Value.text('-');
        smiles1Value.text('-');
        conc0Value.text('-');
        conc1Value.text('-');
        cond0Value.text('-');

        trueValue.removeClass("highlight");
        predValue.removeClass("highlight");
    }}

    // Function to highlight points on the plot by group
    function highlightPlotPointsByGroup(groupId) {{
        const pointCount = gd.data[0].x.length;
        const colors = [];
        const originalColors = Array(pointCount).fill('blue');
        const dimmedColor = 'rgba(200, 200, 200, 0.3)';
        const highlightColor = 'red';

        if (groupId === null) {{
            Plotly.restyle(gd, {{'marker.color': [originalColors]}}, [0]);
        }} else {{
            for (let i = 0; i < pointCount; i++) {{
                // customdata structure: [original_index, group, smiles0, smiles1, conc0, conc1, cond0, ytrue0, ypred0]
                const pointGroup = gd.data[0].customdata[i][1];
                colors.push(pointGroup === groupId ? highlightColor : dimmedColor);
            }}
            Plotly.restyle(gd, {{'marker.color': [colors]}}, [0]);
        }}
    }}

    // Function to highlight table rows by group
    function highlightTableRowsByGroup(groupId) {{
        dataTable.$('tr.highlighted-row').removeClass('highlighted-row'); // Clear all highlights first

        if (groupId !== null) {{
            dataTable.rows(function(idx, rowData, node) {{
                return rowData.group === groupId;
            }}).nodes().to$().addClass('highlighted-row');
        }}
    }}

    // Function to scroll table to a specific row (by original index)
    function scrollToRow(originalIndex) {{
        const row = dataTable.rows(function(idx, rowData, node) {{
            return rowData.original_index === originalIndex;
        }});
        if (row.any()) {{
            const node = row.node();
            if (node) {{
                // Use DataTables' own scroll API if available/needed, or native scrollIntoView
                // scrollIntoView is generally simpler and sufficient
                 node.scrollIntoView({{ behavior: 'smooth', block: 'nearest' }});
            }}
        }}
    }}


    // --- Plotly Hover/Unhover Events ---
    gd.on('plotly_hover', function(eventData) {{
        if (!eventData.points || eventData.points.length === 0) return;

        const point = eventData.points[0];
        if (point.curveNumber !== 0) return;

        // customdata structure: [original_index, group, smiles0, smiles1, conc0, conc1, cond0, ytrue0, ypred0]
        const pointData = point.customdata;
        const originalIndex = pointData[0]; // Get original index from customdata
        const hoveredGroup = pointData[1]; // Get group from customdata

        // Check if the point is already the actively selected/hovered one (e.g., from table click)
        // If the current point index is the same as the one we are about to process, do nothing.
        // This prevents flicker if rapidly hovering over points within the same group or the very same point.
        // Let's rely on the timer and currentHoverPointIndex to manage rapid hovers.

        // Clear previous timer if any
        if (hoverTimer) {{
            clearTimeout(hoverTimer);
        }}

        // Store the index of the point we are now potentially hovering
        currentHoverPointIndex = originalIndex;


        // Set a short delay before taking action
        hoverTimer = setTimeout(function() {{
            // Re-check if this is still the point being hovered after the delay
            // This check is crucial to avoid acting on stale hover events
             if (currentHoverPointIndex === originalIndex) {{
                 // Find the full row data from the DataTable's source data using the original index
                 const rowData = data.find(row => row.original_index === originalIndex);

                 if (rowData) {{
                     updateInfoPanel(rowData); // Update panel using full data

                     highlightPlotPointsByGroup(hoveredGroup); // Highlight plot points

                     // Highlight table rows of the same group and scroll to the hovered point's row
                     highlightTableRowsByGroup(hoveredGroup);
                     scrollToRow(originalIndex);
                 }} else {{
                      console.warn("Row data not found for original index:", originalIndex);
                 }}
             }}
        }}, 75); // Shorter delay (e.g., 75ms) for smoother feel
    }});

    gd.on('plotly_unhover', function() {{
        // Clear the timer and the stored point index immediately on unhover
        if (hoverTimer) {{
            clearTimeout(hoverTimer);
        }}
        currentHoverPointIndex = null; // Clear active hover index

        // After a very short delay, reset the state IF no new hover has started
        // This prevents unhighlighting when moving from one point to another within the delay
        setTimeout(function() {{
             if (currentHoverPointIndex === null) {{ // Only reset if no new point is being hovered
                highlightPlotPointsByGroup(null); // Reset plot colors
                highlightTableRowsByGroup(null); // Clear table highlights
                clearInfoPanel(); // Clear info panel
             }}
        }}, 50); // Very short delay before clearing
    }});


    // --- DataTables Row Click Event ---
    $('#dataTable tbody').on('click', 'tr', function() {{
        // Clear any active hover timer from Plotly immediately
        if (hoverTimer) {{
            clearTimeout(hoverTimer);
            currentHoverPointIndex = null; // Clicking table takes precedence over plot hover
        }}

        const rowData = dataTable.row(this).data(); // Get data for the clicked row

        if (rowData) {{
            const clickedGroup = rowData.group;
            const clickedOriginalIndex = rowData.original_index;

            // Update info panel with clicked row data
            updateInfoPanel(rowData);

            // Highlight corresponding points on the plot by group
            highlightPlotPointsByGroup(clickedGroup);

            // Highlight clicked row(s) in the table (handled by highlightTableRowsByGroup)
            highlightTableRowsByGroup(clickedGroup);

            // Ensure the clicked row is in view (though clicking usually means it is)
            // scrollToRow(clickedOriginalIndex); // Optional, usually the row is already visible

            // Optional: Add a visual indication to the plot point that corresponds to the clicked row,
            // perhaps a different marker style or outline, besides the group highlight.
            // This would require more complex Plotly restyling logic based on index.
            // For now, group highlight is sufficient based on requirements.
        }}
    }});

    // Handle click outside table/plot to clear selections/highlights
    // This requires checking if the click target is outside specific containers.
    // A simpler approach for clearing might be a dedicated button or relying on unhover.
    // Let's add a listener to the document body that clears IF the click wasn't inside the plot or table.
     $('body').on('click', function(event) {{
         // Check if the click target is NOT inside the plot container or the table container
         const $target = $(event.target);
         const isClickInsidePlot = $target.closest('.plot-container').length > 0 || $target.closest('.info-panel').length > 0;
         const isClickInsideTable = $target.closest('.dataTables_wrapper').length > 0; // DataTables wraps the table

         if (!isClickInsidePlot && !isClickInsideTable) {{
             // If clicked outside both, clear everything
             if (hoverTimer) {{ clearTimeout(hoverTimer); }}
             currentHoverPointIndex = null;
             highlightPlotPointsByGroup(null);
             highlightTableRowsByGroup(null);
             clearInfoPanel();
         }}
     }});


    // Initial state: clear info panel and set default plot colors, clear table highlights
    clearInfoPanel();
    highlightPlotPointsByGroup(null);
    highlightTableRowsByGroup(null);


    // Adjust Plotly plot size if window resizes (useful in flexbox layouts)
    // Add a small delay to debounce the resize event
    let resizeTimer;
    $(window).on('resize', function() {{
        clearTimeout(resizeTimer);
        resizeTimer = setTimeout(function() {{
            if (gd && gd.layout.autosize !== true) {{ // Avoid conflicting with Plotly's autosize if enabled
                // Get the current size of the plot container
                 const plotContainer = $(gd).parent('.plot-container');
                 if(plotContainer.length > 0) {{
                     const newWidth = plotContainer.width();
                     // Keep height proportional to width based on initial 8:6 ratio, or keep fixed height
                     // Let's keep the fixed height set in the layout for simplicity with the side panel
                     Plotly.relayout(gd, {{
                         width: newWidth,
                         // height: gd.layout.height // Keep original height
                     }});
                 }}
            }}
        }}, 100); // Debounce delay
    }});


    // Trigger a resize event initially to ensure plot is sized correctly after DataTables renders
     setTimeout(function() {{
         $(window).trigger('resize');
     }}, 10); // Small delay after DOM ready and DataTables init

}}); // End $(document).ready
</script>
"""

# Combine HTML parts
# Format the main HTML structure, injecting the plot div and the JS script.
# Format the JS script, injecting the JSON data string directly.
full_html = html_structure.format(
    plot_div=plot_div_html,
    js_script=js_script_template.format(json_data_placeholder=json_data_string) # Inject the JSON string
)


# === Step 4: 保存为 HTML 文件 ===
with open("viscosity4.html", "w", encoding="utf-8") as f:
    f.write(full_html)

print("HTML文件 'viscosity.html' 已生成成功。")
print("请确保 your_data.csv 文件与脚本在同一目录下。")
print("表格数据已保存到 viscosity.csv。")

HTML文件 'viscosity.html' 已生成成功。
请确保 your_data.csv 文件与脚本在同一目录下。
表格数据已保存到 viscosity.csv。
